# U.S. Census apportionment data in 2010 and 2020

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_latimes as lat
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
from vega_datasets import data

states = alt.topo_feature(data.us_10m.url, "states")

In [4]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Get the data from the U.S. Census

In [5]:
# https://www.census.gov/data/tables/2010/dec/2010-apportionment-data.html

In [6]:
url2010 = "https://www2.census.gov/programs-surveys/decennial/2010/data/apportionment/apport2010-table1.xls"

In [7]:
df_2010 = pd.read_excel(url2010, encoding="Latin-1", skiprows=10, skipfooter=6)

In [8]:
df_2010.columns = ["state", "population", "drop1", "seats", "drop2", "seats_change"]

In [9]:
df_2010.drop(["drop1", "drop2"], axis=1, inplace=True)

In [10]:
df_2010.tail(10)

,state,population,seats,seats_change
40,South Dakota,819761,1,0
41,Tennessee,6375431,9,0
42,Texas,25268418,36,4
43,Utah,2770765,4,1
44,Vermont,630337,1,0
45,Virginia,8037736,11,0
46,Washington,6753369,10,1
47,West Virginia,1859815,3,0
48,Wisconsin,5698230,8,0
49,Wyoming,568300,1,0


In [11]:
df_2010["year"] = "2010"

In [12]:
url2020 = "https://www2.census.gov/programs-surveys/decennial/2020/data/apportionment/apportionment-2020-table01.xlsx"

In [13]:
df_2020 = pd.read_excel(url2020, encoding="Latin-1", skiprows=3, skipfooter=2)

In [14]:
df_2020.tail()

,STATE,"APPORTIONMENT POPULATION \n(APRIL 1, 2020)",NUMBER OF APPORTIONED REPRESENTATIVES BASED ON \n2020 CENSUS2,CHANGE FROM \n2010 CENSUS APPORTIONMENT
45,Virginia,8654542,11,0
46,Washington,7715946,10,0
47,West Virginia,1795045,2,-1
48,Wisconsin,5897473,8,0
49,Wyoming,577719,1,0


In [15]:
df_2020.columns = ["state", "population", "seats", "seats_change"]

In [16]:
df_2020["year"] = "2020"

In [17]:
df = pd.concat([df_2010, df_2020])

In [18]:
df_2020.head()

,state,population,seats,seats_change,year
0,Alabama,5030053,7,0,2020
1,Alaska,736081,1,0,2020
2,Arizona,7158923,9,0,2020
3,Arkansas,3013756,4,0,2020
4,California,39576757,52,-1,2020


---

## Historical apportionment

### Grab gnarly Census table

In [19]:
hist_df = pd.read_excel(
    "https://www2.census.gov/programs-surveys/decennial/2020/data/apportionment/apportionment-2020-tableC2.xlsx",
    skiprows=4,
    skipfooter=3,
)

### Awesome headers and data structure, y'all

In [20]:
hist_df.tail()

,Unnamed: 0,2020: Apportionment population,2020: \nNumber of representatives,2020: \nSeat change,2020: Average persons per representative,2010: Apportionment population,2010: \nNumber of representatives,2010: \nSeat change,2010: Average persons per representative,Unnamed: 9,2000: Apportionment population,2000: \nNumber of representatives,2000: \nSeat change,2000: Average persons per representative,1990: Apportionment population,1990: \nNumber of representatives,1990: \nSeat change,1990: Average persons per representative,Unnamed: 18,1980: Apportionment population,1980: \nNumber of representatives,1980: \nSeat change,1980: Average persons per representative,1970: Apportionment population,1970: \nNumber of representatives,...,1960: Apportionment population,1960: \nNumber of representatives,1960: \nSeat change,1960: Average persons per representative,1950: Apportionment population,1950: \nNumber of representatives,1950: \nSeat change,1950: Average persons per representative,Unnamed: 36,1940: Apportionment population,1940: \nNumber of representatives,1940: \nSeat change,1940: Average persons per representative,1930: Apportionment population,1930: \nNumber of representatives,1930: \nSeat change,1930: Average persons per representative,Unnamed: 45,1920: Apportionment population,1920: \nNumber of representatives,1920: Average persons per representative,1910: Apportionment population,1910: \nNumber of representatives,1910: \nSeat change\n(since 1900),1910: Average persons per representative
45,Virginia,8654542,11,0,786777,8037736,11,0,730703,Virginia,7100702,11,0,645518,6216568,11,1,565143,Virginia,5346279,10,0,534628,4690742,10,...,3966949,10,0,396695,3318680,10,1,331868,Virginia,2677773,9,0,297530,2421829,9,-1,269092,Virginia,2309187,10,230919,2061612,10,0,206161
46,Washington,7715946,10,0,771595,6753369,10,1,675337,Washington,5908684,9,0,656520,4887941,9,1,543105,Washington,4130163,8,1,516270,3443487,7,...,2853214,7,0,407602,2378963,7,1,339852,Washington,1736191,6,0,289365,1552423,6,1,258737,Washington,1354596,5,270919,1140134,5,2,228027
47,West Virginia,1795045,2,-1,897523,1859815,3,0,619938,West Virginia,1813077,3,0,604359,1801625,3,-1,600542,West Virginia,1949644,4,0,487411,1763331,4,...,1860421,5,-1,372084,2005552,6,0,334259,West Virginia,1901974,6,0,316996,1729199,6,0,288200,West Virginia,1463701,6,243950,1221119,6,1,203520
48,Wisconsin,5897473,8,0,737184,5698230,8,0,712279,Wisconsin,5371210,8,-1,671401,4906745,9,0,545194,Wisconsin,4705335,9,0,522815,4447013,9,...,3951777,10,0,395178,3434575,10,0,343458,Wisconsin,3137587,10,0,313759,2931721,10,-1,293172,Wisconsin,2631305,11,239210,2332853,11,0,212078
49,Wyoming,577719,1,0,577719,568300,1,0,568300,Wyoming,495304,1,0,495304,455975,1,0,455975,Wyoming,470816,1,0,470816,335719,1,...,330066,1,0,330066,290529,1,0,290529,Wyoming,250742,1,0,250742,223630,1,0,223630,Wyoming,193487,1,193487,144658,1,0,144658


### Rename the columns

In [21]:
hist_df.columns = [
    "name",
    "2020pop",
    "2020reps",
    "2020change",
    "2020pop_per_rep",
    "2010pop",
    "2010reps",
    "2010change",
    "2010pop_per_rep",
    "name2000",
    "2000pop",
    "2000reps",
    "2000change",
    "2000pop_per_rep",
    "1990pop",
    "1990reps",
    "1990change",
    "1990pop_per_rep",
    "name1980",
    "1980pop",
    "1980reps",
    "1980change",
    "1980pop_per_rep",
    "1970pop",
    "1970reps",
    "1970change",
    "1970pop_per_rep",
    "name1960",
    "1960pop",
    "1960reps",
    "1960change",
    "1960pop_per_rep",
    "1950pop",
    "1950reps",
    "1950change",
    "1950pop_per_rep",
    "name1940",
    "1940pop",
    "1940reps",
    "1940change",
    "1940pop_per_rep",
    "1930pop",
    "1930reps",
    "1930change",
    "1930pop_per_rep",
    "name1920",
    "1920pop",
    "1920reps",
    "1920pop_per_rep",
    "1910pop",
    "1910reps",
    "1910change",
    "1910pop_per_rep",
]

### No change for 1920 in the table, so let's add it

In [22]:
hist_df["1920change"] = 0

### Slim down the dataframe

In [23]:
hist_df = hist_df[
    [
        "name",
        "2020pop",
        "2020reps",
        "2020change",
        "2020pop_per_rep",
        "2010pop",
        "2010reps",
        "2010change",
        "2010pop_per_rep",
        "2000pop",
        "2000reps",
        "2000change",
        "2000pop_per_rep",
        "1990pop",
        "1990reps",
        "1990change",
        "1990pop_per_rep",
        "1980pop",
        "1980reps",
        "1980change",
        "1980pop_per_rep",
        "1970pop",
        "1970reps",
        "1970change",
        "1970pop_per_rep",
        "1960pop",
        "1960reps",
        "1960change",
        "1960pop_per_rep",
        "1950pop",
        "1950reps",
        "1950change",
        "1950pop_per_rep",
        "1940pop",
        "1940reps",
        "1940change",
        "1940pop_per_rep",
        "1930pop",
        "1930reps",
        "1930change",
        "1930pop_per_rep",
        "1920pop",
        "1920reps",
        "1920change",
        "1920pop_per_rep",
        "1910pop",
        "1910reps",
        "1910change",
        "1910pop_per_rep",
    ]
]

### Melt the dataframe

In [24]:
hist_df_melt = pd.melt(
    hist_df,
    id_vars="name",
    value_vars=[
        "2020pop",
        "2020reps",
        "2020change",
        "2020pop_per_rep",
        "2010pop",
        "2010reps",
        "2010change",
        "2010pop_per_rep",
        "2000pop",
        "2000reps",
        "2000change",
        "2000pop_per_rep",
        "1990pop",
        "1990reps",
        "1990change",
        "1990pop_per_rep",
        "1980pop",
        "1980reps",
        "1980change",
        "1980pop_per_rep",
        "1970pop",
        "1970reps",
        "1970change",
        "1970pop_per_rep",
        "1960pop",
        "1960reps",
        "1960change",
        "1960pop_per_rep",
        "1950pop",
        "1950reps",
        "1950change",
        "1950pop_per_rep",
        "1940pop",
        "1940reps",
        "1940change",
        "1940pop_per_rep",
        "1930pop",
        "1930reps",
        "1930change",
        "1930pop_per_rep",
        "1920pop",
        "1920reps",
        "1920change",
        "1920pop_per_rep",
        "1910pop",
        "1910reps",
        "1910change",
        "1910pop_per_rep",
    ],
)

### Clean up and add columns for year and category

In [25]:
hist_df_melt["year"] = hist_df_melt["variable"].str[:4]

In [26]:
hist_df_melt["category"] = hist_df_melt["variable"].str.replace("\d+", "", regex=True)

In [27]:
hist_df_melt.drop(["variable"], axis=1, inplace=True)

### Change in number of representatives, by state and year 

In [28]:
change = hist_df_melt[hist_df_melt["category"] == "change"]

In [29]:
# small_states = [
#     "Alaska",
#     "Wyoming",
#     "Montana",
#     "North Dakota",
#     "South Dakota",
#     "Vermont",
#     "Delaware",
# ]

In [30]:
mini_multiples_states = (
    alt.Chart(change)
    .mark_bar()
    .encode(
        y=alt.Y(
            "value:Q",
            title=" ",
            axis=alt.Axis(
                tickSize=0,
                domainOpacity=0,
                tickCount=2,
                offset=4,
                gridWidth=0.6,
                gridColor="#dddddd",
            ),
        ),
        x=alt.X(
            "year:O",
            title=" ",
            axis=alt.Axis(
                tickSize=0,
                domainOpacity=0,
                tickCount=3,
                offset=4,
                gridWidth=0.6,
                gridColor="#dddddd",
                values=["1910", "2020"],
            ),
        ),
        facet=alt.Facet(
            "name:O",
            columns=10,
            title=" ",
            header=alt.Header(labelFontSize=15, labelFont="Benton Gothic"),
        ),
        color=alt.condition(
            alt.datum.value > 0,
            alt.value("steelblue"),  # The positive color
            alt.value("red"),  # The negative color
        ),
    )
    .properties(width=80, height=80, title="Change in representatives by Census year")
    .configure_view(strokeOpacity=0)
    .configure_legend(orient="top", symbolType="square")
    .configure_axis(labelFontSize=12)
)
mini_multiples_states

alt.Chart(...)

In [31]:
mini_multiples_states.save("visuals/mini_multiples_states.png", scale_factor=1)

### Alternative chart forms

In [32]:
alt.Chart(change).mark_rect().encode(
    x="year:O",
    y="name:O",
    color=alt.condition(
        alt.datum.value < 0,
        alt.value("red"),  # The positive color
        alt.value("lightgray"),  # The negative color
    ),
).properties(height=600, width=300, title="When states lost representation")

alt.Chart(...)

### What's the sum of seats by state?

In [33]:
hist_df_melt.head()

,name,value,year,category
0,Alabama,5030053,2020,pop
1,Alaska,736081,2020,pop
2,Arizona,7158923,2020,pop
3,Arkansas,3013756,2020,pop
4,California,39576757,2020,pop


In [34]:
hist_df_melt["value"] = hist_df_melt["value"].str.replace("X", "0", regex=False)

In [35]:
hist_df_melt[hist_df_melt["value"] == "X"]

,name,value,year,category


In [36]:
hist_df_melt.head()

,name,value,year,category
0,Alabama,NaN,2020,pop
1,Alaska,NaN,2020,pop
2,Arizona,NaN,2020,pop
3,Arkansas,NaN,2020,pop
4,California,NaN,2020,pop


### Rhode Island

In [37]:
hist_df_ri = hist_df_melt[hist_df_melt["name"] == "Rhode Island"].copy()

In [38]:
hist_df_ri[hist_df_ri["category"] == "change"]

,name,value,year,category
138,Rhode Island,NaN,2020,change
338,Rhode Island,NaN,2010,change
538,Rhode Island,NaN,2000,change
738,Rhode Island,NaN,1990,change
938,Rhode Island,NaN,1980,change
1138,Rhode Island,NaN,1970,change
1338,Rhode Island,NaN,1960,change
1538,Rhode Island,NaN,1950,change
1738,Rhode Island,NaN,1940,change
1938,Rhode Island,NaN,1930,change


### Just California

In [39]:
hist_df_ca = hist_df_melt[hist_df_melt["name"] == "California"].copy()

In [40]:
hist_df_ca_pivot = pd.pivot_table(
    hist_df_ca,
    values="value",
    index=["name", "year"],
    columns="category",
    aggfunc=sum,
    fill_value=0,
).reset_index()

In [41]:
hist_df_ca_pivot.head(12)

category,name,year,change,pop,pop_per_rep,reps
0,California,1910,0,0,0,0
1,California,1920,0,0,0,0
2,California,1930,0,0,0,0
3,California,1940,0,0,0,0
4,California,1950,0,0,0,0
5,California,1960,0,0,0,0
6,California,1970,0,0,0,0
7,California,1980,0,0,0,0
8,California,1990,0,0,0,0
9,California,2000,0,0,0,0


In [42]:
bars = alt.Chart(hist_df_ca_pivot).mark_bar().encode(y="change:Q", x="year:O")

text = bars.mark_text(
    align="left",
    baseline="middle",
    dy=-10,  # Nudges text to right so it doesn't appear on top of the bar
    dx=-3,
).encode(text="change:Q")

(bars + text).properties(height=400)

alt.LayerChart(...)

In [43]:
lines = alt.Chart(hist_df_ca_pivot).mark_line().encode(x="year:O", y="pop:Q")

(lines).properties(height=400)

alt.Chart(...)

---

## Geography

### States map

In [44]:
state_geo = gpd.read_file("raw/states.geojson")
state_geo.columns = state_geo.columns.str.lower()

### Add A.P. states

In [45]:
ap_states = pd.read_csv("raw/ap_states.csv")

In [46]:
states_merge = state_geo.merge(ap_states, left_on="stusps", right_on="usps")

### Clean up

In [47]:
states_merge.drop(
    ["name_x", "stusps", "statefp", "statens", "affgeoid", "lsad", "aland", "awater"],
    axis=1,
    inplace=True,
)

In [48]:
states_merge.rename(columns={"name_y": "name"}, inplace=True)

In [49]:
states_merge = states_merge[["geoid", "name", "usps", "ap", "geometry"]]

In [50]:
states_merge.head()

,geoid,name,usps,ap,geometry
0,23,Maine,ME,Maine,"MULTIPOLYGON (((-68.92401 43.88541, -68.87478 ..."
1,15,Hawaii,HI,Hawaii,"MULTIPOLYGON (((-156.04965 19.78045, -156.0062..."
2,04,Arizona,AZ,Ariz.,"MULTIPOLYGON (((-114.79968 32.59362, -114.8093..."
3,05,Arkansas,AR,Ark.,"MULTIPOLYGON (((-94.61792 36.49941, -94.36120 ..."
4,10,Delaware,DE,Del.,"MULTIPOLYGON (((-75.77379 39.72220, -75.75323 ..."


---

## Export 

In [51]:
df_2010.to_csv("processed/apportionment/apportionment_2010.csv", index=False)

In [52]:
df_2020.to_csv("processed/apportionment/apportionment_2020.csv", index=False)

In [53]:
df.to_csv("processed/apportionment/apportionment_2010_2020.csv", index=False)

In [54]:
hist_df_ca_pivot.to_csv(
    "processed/apportionment/apportionment_in_ca_1910_2020.csv", index=False
)